# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Reading the data

In [ ]:
# Path to file
mrt_file_path = '/kaggle/input/datasets/kathirvelsasikumar/mrt-stations/mrt_stations.csv'

# Read data
mrt_data = pd.read_csv(mrt_file_path, index_col=0)

# Brief look at the data
print(mrt_data.head())
print(mrt_data.shape)

# Exploration of data

In [ ]:
# Columns
print("\nColumn names:")
print(mrt_data.columns.tolist())

# Check for missing values
print("\nMissing values:")
print(mrt_data.isnull().sum())

# Summary statistics
print("\nBasic stats:")
print(mrt_data.describe())

# Feature Engineering

In [ ]:
# Convert opening date to datetime
mrt_data['opening'] = pd.to_datetime(mrt_data['opening'])
mrt_data['opening_year'] = mrt_data['opening'].dt.year
mrt_data['station_age'] = 2026 - mrt_data['opening_year']

# Is it an interchange station?
# Interchange stations appear multiple times (different lines at same location)
# Count how many times each station name appears
station_counts = mrt_data['station_name'].value_counts()
mrt_data['is_interchange'] = mrt_data['station_name'].map(station_counts)
mrt_data['is_interchange'] = (mrt_data['is_interchange'] > 1).astype(int)
mrt_data['num_lines'] = mrt_data['station_name'].map(station_counts)

# Create dummy variables for regions
region_dummies = pd.get_dummies(mrt_data['region_ura'], prefix='region')
mrt_data = pd.concat([mrt_data, region_dummies], axis=1)

# Create features for CBD proximity
cbd_areas = ['DOWNTOWN CORE', 'ORCHARD', 'MUSEUM', 'MARINA SOUTH', 'STRAITS VIEW']
mrt_data['near_cbd'] = mrt_data['planning_area_ura'].isin(cbd_areas).astype(int)

# Nightlife/entertainment areas
nightlife = ['ORCHARD', 'CLARKE QUAY', 'RIVER VALLEY', 'BUGIS']
mrt_data['nightlife_zone'] = mrt_data['planning_area_ura'].isin(nightlife).astype(int)

# Count stations per area (density)
area_station_counts = mrt_data['planning_area_ura'].value_counts()
mrt_data['area_density'] = mrt_data['planning_area_ura'].map(area_station_counts)

print("\nNew features created:")
print(f"- station_age: {mrt_data['station_age'].min()} to {mrt_data['station_age'].max()} years")
print(f"- is_interchange: {mrt_data[mrt_data['is_interchange']==1]['station_name'].nunique()} unique interchange stations")
print(f"- interchange entries: {mrt_data['is_interchange'].sum()} (counting all line instances)")
print(f"- near_cbd: {mrt_data['near_cbd'].sum()} stations")
print(f"- nightlife_zone: {mrt_data['nightlife_zone'].sum()} stations")

# Show examples
print("\nSample interchange stations:")
interchange_examples = mrt_data[mrt_data['is_interchange']==1].groupby('station_name')['line'].apply(list).head(5)
for station, lines in interchange_examples.items():
    print(f"  {station}: {len(lines)} lines")
print()

# Create target variable (simulated incident risk)

Note: In a real scenario, I'd have actual incident data from SPF. However , i cannot access actual SPF incident data due to confidentiality and security restrictions hence , to abid by ethical guidelands, I would create a realistic simulation model that generates incident data based on known risk factors.


In [ ]:
# Varibles
def generate_incident_risk(row):
    """Generate risk score based on station characteristics"""
    base_risk = 5  # ← LOWER baseline
    
    # Interchange stations have more incidents (but with variation)
    if row['is_interchange']:
        base_risk += np.random.uniform(8, 18)  # ← RANDOM RANGE 8-18
    
    # CBD areas have higher risk (but not all the same)
    if row['near_cbd']:
        base_risk += np.random.uniform(5, 15)  # ← RANDOM RANGE 5-15
    
    # Nightlife zones peak at night
    if row['nightlife_zone']:
        base_risk += np.random.uniform(3, 12)  # ← RANDOM RANGE 3-12
    
    # Older stations might have more issues (not always)
    if row['station_age'] > 20:
        base_risk += np.random.uniform(0, 8)  # ← RANDOM RANGE 0-8
    
    # High density areas (variable crowding effects)
    if row['area_density'] > 5:
        base_risk += np.random.uniform(2, 10)  # ← RANDOM RANGE 2-10
    
    # Add significant randomness to simulate unpredictable factors
    noise = np.random.normal(0, 8)  # ← BIGGER noise (was 3, now 8)
    
    # Some stations just have random spikes (NEW!)
    if np.random.random() < 0.1:  # 10% chance
        noise += np.random.uniform(5, 15)  # Random spike
    
    return max(0, base_risk + noise)

# Generate risk scores
mrt_data['monthly_incidents'] = mrt_data.apply(generate_incident_risk, axis=1)

# Create binary classification target (high risk = 1, low risk = 0)
risk_threshold = mrt_data['monthly_incidents'].median()
mrt_data['high_risk'] = (mrt_data['monthly_incidents'] > risk_threshold).astype(int)

print(f"\nTarget variable created:")
print(f"- Average incidents per station: {mrt_data['monthly_incidents'].mean():.1f}")
print(f"- High risk stations: {mrt_data['high_risk'].sum()}")
print(f"- Low risk stations: {(1-mrt_data['high_risk']).sum()}")

# Select features for modeling

In [ ]:
# Choose which columns to use as numerical features

feature_cols = [
    'station_age', 
    'is_interchange', 
    'near_cbd', 
    'nightlife_zone',
    'area_density',
    'latitude',
    'longitude'
] + [col for col in mrt_data.columns if col.startswith('region_')]

# Remove any non-numeric columns
X = mrt_data[feature_cols].copy()
# Keep only numeric columns
X = X.select_dtypes(include=[np.number])

y = mrt_data['high_risk'].copy()

print(f"\nFeatures selected: {len(X.columns)}")
print("Feature list:", X.columns.tolist())

# Split data into training and validation sets

In [ ]:
# Break off validation set from training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1, test_size=0.25)

print(f"\nTraining set size: {len(train_X)}")
print(f"Validation set size: {len(val_X)}")

# Build Model

In [ ]:
# Define the model
# Using Random Forest
rf_model = GradientBoostingClassifier(n_estimators=100, random_state=1, max_depth=5)

# Fit model
rf_model.fit(train_X, train_y)

# Make predictions
train_preds = rf_model.predict(train_X)
val_preds = rf_model.predict(val_X)

print("MODEL PERFORMANCE")

# Check accuracy
train_accuracy = (train_preds == train_y).mean()
val_accuracy = (val_preds == val_y).mean()

print(f"\nTraining Accuracy: {train_accuracy:.3f}")
print(f"Validation Accuracy: {val_accuracy:.3f}")

# Detailed classification report
print("\nValidation Set Performance:")
print(classification_report(val_y, val_preds, 
                          target_names=['Low Risk', 'High Risk']))


# Feature Importance

In [ ]:
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

# Make Predictions on Full Dataset

In [ ]:
# Probability predictions for all stations
mrt_data['risk_probability'] = rf_model.predict_proba(X)[:, 1]
mrt_data['predicted_high_risk'] = rf_model.predict(X)

# Create risk categories
mrt_data['risk_category'] = pd.cut(mrt_data['risk_probability'], 
                                     bins=[0, 0.3, 0.7, 1.0],
                                     labels=['Low', 'Medium', 'High'])

print("RISK ASSESSMENT SUMMARY")

print(mrt_data['risk_category'].value_counts().sort_index())


# Identify High Priority Stations

In [ ]:
# Sort by risk probability
high_priority = mrt_data.nlargest(20, 'risk_probability')[
    ['station_name', 'line', 'planning_area_ura', 'is_interchange', 
     'near_cbd', 'risk_probability', 'risk_category']
]

print("\n" + "="*60)
print("TOP 20 STATIONS REQUIRING INCREASED PATROL")
print("="*60)
print(high_priority.to_string())

# Deployment Recommendation (Theorectical)

**Assumptions** :  
- 3-man patrol groups cover multiple nearby stations
- Each group gets 2 × 1-hour breaks during their shift
- Groups prioritize high-risk stations within their region

In [ ]:
# Calculate officer allocation per station
total_officers = 70  # Average per shift
mrt_data['recommended_officers'] = (mrt_data['risk_probability'] * total_officers / mrt_data['risk_probability'].sum()).round()

# Ensure at least 1 officer at high risk stations
mrt_data.loc[mrt_data['risk_category'] == 'High', 'recommended_officers'] = \
    mrt_data.loc[mrt_data['risk_category'] == 'High', 'recommended_officers'].clip(lower=1)



# Define regions - combine Central and South (CBD areas are high-risk)
regions_mapping = {
    'NORTH REGION': 'NORTH REGION',
    'SOUTH REGION': 'CENTRAL REGION (CBD)',  # South is part of CBD
    'EAST REGION': 'EAST REGION',
    'WEST REGION': 'WEST REGION',
    'CENTRAL REGION': 'CENTRAL REGION (CBD)',
    'NORTH-EAST REGION': 'NORTH-EAST REGION'
}

# Map regions in data
mrt_data['patrol_region'] = mrt_data['region_ura'].map(regions_mapping)

# Define 4 patrol groups
patrol_regions = ['NORTH REGION', 'CENTRAL REGION (CBD)', 'EAST REGION', 'WEST REGION']

# Create 4 patrol groups (one per region)
for region_idx, region in enumerate(patrol_regions, 1):
    print(f"\n{'='*70}")
    print(f"PATROL GROUP {region_idx}: {region}")
    print(f"{'='*70}")
    
    # Get stations in this region, sorted by risk
    region_stations = mrt_data[mrt_data['patrol_region'] == region].copy()
    
    if len(region_stations) == 0:
        print(f"   No stations in {region}")
        continue
    
    # Get unique stations sorted by risk
    unique_stations = region_stations.groupby('station_name').agg({
        'risk_probability': 'mean',
        'is_interchange': 'first',
        'num_lines': 'first',
        'planning_area_ura': 'first',
        'latitude': 'first',
        'longitude': 'first'
    }).sort_values('risk_probability', ascending=False)
    
    # Get top 5 high-risk stations in this region
    top_stations = unique_stations.head(5)

    
    print(f"{'Priority':<10} {'Station':<25} {'Type':<20} {'Risk':<10} {'Area'}")
    print("-" * 70)
    
    for idx, (station_name, row) in enumerate(top_stations.iterrows(), 1):
        station_type = f"Interchange ({int(row['num_lines'])} lines)" if row['is_interchange'] else "Regular"
        risk_score = f"{row['risk_probability']:.3f}"
        
        print(f"#{idx:<9} {station_name:<25} {station_type:<20} {risk_score:<10} {row['planning_area_ura']}")
    
    # Calculate distance clustering 
    if len(top_stations) >= 2:
        print(f"\n Patrol Route Suggestion:")
        print(f"   Start → ", end="")
        for i, station in enumerate(top_stations.head(3).index):
            if i < len(top_stations.head(3)) - 1:
                print(f"{station} → ", end="")
            else:
                print(f"{station}")
        print(f"   (Covering stations in close proximity)")